In [3]:
from tkinter import *
import sqlite3
from tkinter import messagebox
import datetime
from tkinter import END
window = Tk()
window.geometry("900x900")
window.title(" ")
window.config(bg="black")
conn = sqlite3.connect("datas.db")
cursor = conn.cursor()

# Create tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        username TEXT NOT NULL UNIQUE,
        password TEXT NOT NULL,
        address TEXT NOT NULL,
        phone_number TEXT NOT NULL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS admins (
        id INTEGER PRIMARY KEY,
        username TEXT NOT NULL UNIQUE,
        password TEXT NOT NULL,
        address TEXT NOT NULL,
        phone_number TEXT NOT NULL
    )
''')

cursor.execute('''CREATE TABLE IF NOT EXISTS online_record (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    height REAL,
    weight REAL,
    temp REAL,
    BP REAL,
    problem TEXT NOT NULL,
    date DATE,
    FOREIGN KEY (user_id) REFERENCES users(id)
)''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS doctor_feedback (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        feedback TEXT NOT NULL,
        date DATE,
        FOREIGN KEY (user_id) REFERENCES users(id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS appointments (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        appointment_date TEXT NOT NULL,
        appointment_time TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id)
    )
''')

conn.commit()

# Current user variable
current_user = ""

def login_user():
    username = username_entry.get()
    password = password_entry.get()
    cursor.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
    user = cursor.fetchone()
    if user:
        global current_user
        current_user = username
        messagebox.showinfo("Success", "Login Successful!")
        switch_to_user_app()
    else:
        messagebox.showerror("Error", "Invalid username or password")
    username_entry.delete(0, END)
    password_entry.delete(0, END)

def login_admin():
    username = admin_login_username_entry.get()
    password = admin_login_password_entry.get()
    cursor.execute("SELECT * FROM admins WHERE username=? AND password=?", (username, password))
    admin = cursor.fetchone()
    if admin:
        messagebox.showinfo("Success", "Doctor Login Successful!")
        switch_to_admin_panel()
    else:
        messagebox.showerror("Error", "Invalid username or password")
    admin_login_username_entry.delete(0, END)
    admin_login_password_entry.delete(0, END)

def register_user():
    username = user_username_entry.get()
    password = user_password_entry.get()
    address = user_address_entry.get()
    phone_number = user_phone_entry.get()
    if username and password and address and phone_number:
        cursor.execute("INSERT INTO users (username, password, address, phone_number) VALUES (?, ?, ?, ?)",
                       (username, password, address, phone_number))
        conn.commit()
        messagebox.showinfo("Success", "Registered Successfully!")
        switch_to_user_login()
    else:
        messagebox.showerror("Error", "All fields are required!")

def register_admin():
    username = admin_username_entry.get()
    password = admin_password_entry.get()
    address = admin_address_entry.get()
    phone_number = admin_phone_entry.get()
    if username and password and address and phone_number:
        cursor.execute("INSERT INTO admins (username, password, address, phone_number) VALUES (?, ?, ?, ?)",
                       (username, password, address, phone_number))
        conn.commit()
        messagebox.showinfo("Success", "Registered Successfully!")
        switch_to_admin_login()
    else:
        messagebox.showerror("Error", "All fields are required!")
    
def save_online():
    height = user_height_entry.get()
    weight = user_weight_entry.get()
    temp = user_temp_entry.get()
    blood = user_BP_entry.get()
    problem = pro_text.get("1.0", "end-1c")
    date = datetime.datetime.today().date()  # Get current date correctly
    user_id = get_user_id(current_user)
    
    if user_id and height and weight and temp and blood and problem:
        cursor.execute("INSERT INTO online_record (user_id, height, weight, temp, BP, problem, date) VALUES (?, ?, ?, ?, ?, ?, ?)",
                       (user_id, height, weight, temp, blood, problem, date))
        conn.commit()  # Commit the changes to the database
        messagebox.showinfo("Success", "Patient Details Saved")
    else:
        messagebox.showerror("Error", "All fields are required!")  # Show error if any information is missing

def fetch_health_details():
    username = user_name_entry.get()
    cursor.execute("SELECT * FROM users WHERE username=?", (username,))
    user = cursor.fetchone()
    if user:
        user_id = user[0]
        cursor.execute("SELECT * FROM online_record WHERE user_id=?", (user_id,))
        records = cursor.fetchall()
        if records:
            for record in records:
                health_details_text.insert(END,f"Date: {record[7]}\nHeight: {record[2]}, Weight: {record[3]},Temp: {record[4]},BP: {record[5]}, Problem: {record[6]}\n")
            feedback_label.grid(row=4, column=0, pady=5)
            feedback_text.grid(row=5, column=0, pady=5)
            submit_feedback_button.grid(row=6, column=0, pady=5)
        else:
            health_details_text.insert(END, "No  records available.")
    else:
        messagebox.showerror("Error", "User not found.")   

def submit_feedback():
    try:
        feedback = feedback_text.get("1.0", END).strip()  # Trim any extra whitespace
        user_id = get_user_id(current_user)
        
        if user_id and feedback:
            cursor.execute("INSERT INTO doctor_feedback (user_id, feedback, date) VALUES (?, ?, ?)",
                           (user_id, feedback, datetime.date.today()))
            conn.commit()
            messagebox.showinfo("Feedback Submitted", "Feedback submitted successfully!")
        else:
            messagebox.showerror("Error", "Failed to submit feedback. Please make sure all fields are filled.")
        
        # Clear feedback text box after submission
        feedback_text.delete("1.0", END)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")
    
def fetch_patient_feedback():
    user_id = get_user_id(current_user)
    if user_id:
        cursor.execute("SELECT * FROM doctor_feedback WHERE user_id=?", (user_id,))
        feedback_records = cursor.fetchall()
        if feedback_records:
            feedback_text = "\n".join([f"{record[3]}: {record[2]}" for record in feedback_records])
            messagebox.showinfo(" Doctor Feedback", feedback_text)
        else: messagebox.showinfo(" Doctor Feedback", "No feedback available.")
    else:
        messagebox.showerror("Error", "User not found.")


def book_appointment():
    date = appointment_date_entry.get()
    time = appointment_time_entry.get()
    user_id = get_user_id(current_user)
    # Enhanced debugging to see what's going wrong
    print(f"Booking attempt: Date={date}, Time={time}, User ID={user_id}")
    if not date:
        print("Error: Date field is empty")
    if not time:
        print("Error: Time field is empty")
    if not user_id:
        print("Error: User ID is None")
    if user_id and date and time:
        # Check if the appointment slot is already booked
        cursor.execute("SELECT COUNT(*) FROM appointments WHERE appointment_date = ? AND appointment_time = ?", (date, time))
        result = cursor.fetchone()
        if result[0] > 0:
            messagebox.showerror("Error", "This appointment slot is already booked. Please choose a different time.")
        else:
            cursor.execute("INSERT INTO appointments (user_id, appointment_date, appointment_time) VALUES (?, ?, ?)",
                           (user_id, date, time))
            conn.commit()
            messagebox.showinfo("Success", "Appointment Booked Successfully!")
    else:
        messagebox.showerror("Error", "All fields are required!")
        
        
def view_appointments():
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")  # Get current date in the correct format
    cursor.execute("""
        SELECT appointments.*, users.username 
        FROM appointments 
        JOIN users ON appointments.user_id = users.id 
        WHERE appointments.appointment_date = ?
    """, (current_date,))
    appointments = cursor.fetchall()
    if appointments:
        appointment_text.delete('1.0', END)
        for appointment in appointments:
            appointment_text.insert(END,
                                     f"Patient: {appointment[-1]}, Date: {appointment[2]}, Time: {appointment[3]}\n\n")
    else:
        appointment_text.delete('1.0', END)  # Clear the text widget if no appointments are found
        appointment_text.insert(END, "No appointments found.")


def get_user_id(username):
    cursor.execute("SELECT * FROM users WHERE username=?", (username,))
    user = cursor.fetchone()
    if user:
        print(f"User ID for {username}: {user[0]}")  # Debugging line
        return user[0]
    else:
        print(f"No user found for {username}")  # Debugging line
        return None

def homepage():
    home.grid()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_user_login():
    home.grid_remove()
    user_login.grid()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_admin_login():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_user_registration():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_admin_registration():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_user_app():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_admin_panel():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_appointment_frame():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_appointment_view():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid()
    frame10.grid_remove()
    frame11.grid_remove()

def switch_to_frame10():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame10.grid()
    frame11.grid_remove()
    
def switch_to_frame11():
    home.grid_remove()
    user_login.grid_remove()
    user_registration.grid_remove()
    admin_login.grid_remove()
    admin_registration.grid_remove()
    user_details.grid_remove()
    admin_dashboard.grid_remove()
    appointment_frame.grid_remove()
    appointment_view_frame.grid_remove()
    frame11.grid()
    frame10.grid_remove()

home = Frame(window)
user_login = Frame(window)
user_registration = Frame(window)
user_details = Frame(window)
admin_login = Frame(window)
admin_registration = Frame(window)
admin_dashboard = Frame(window)
appointment_frame = Frame(window)
appointment_view_frame = Frame(window)
frame10 = Frame(window)
frame11 = Frame(window)


frames_list = [window, home, user_login, user_registration, admin_login, admin_registration, user_details, admin_dashboard,appointment_frame,appointment_view_frame,frame10,frame11]

for frame in frames_list:
    frame.config(bg="black")
    frame.grid_rowconfigure(0, weight=1)
    frame.grid_columnconfigure(0, weight=1)

#Home Page
home_heading = Label(home, text="Online Medical Checkup", font=("Cooper Black", 40), fg="white", bg="black")
home_heading.grid(row=0, column=0, columnspan=2, padx=5, pady=25)
user_login_button = Button(home, text="Patient Login", font=("Goudy Old Style", 18), fg="white", bg="black", command=switch_to_user_login)
user_login_button.grid(row=1, column=0, padx=20, pady=10, sticky='e')
admin_login_button = Button(home, text="Doctor Login", font=("Goudy Old Style", 18), fg="white", bg="black", command=switch_to_admin_login)
admin_login_button.grid(row=1, column=1, padx=20, pady=10, sticky='w')
# Configure the columns to adjust their weight for centering the buttons
home.grid_columnconfigure(0, weight=1)
home.grid_columnconfigure(1, weight=1)

#user login page
login_heading_label = Label(user_login, text="Patient Login", font=("Cooper Black", 40), fg="white", bg="black")
login_heading_label.grid(row=0, column=0, padx=20, pady=20, columnspan=2)
username_label = Label(user_login, text="Username:", font=("Georgia", 20), fg="white", bg="black")
username_label.grid(row=1, column=0, padx=20, pady=10)
username_entry = Entry(user_login, font=("Arial", 20))
username_entry.grid(row=1, column=1, padx=20, pady=10)
password_label = Label(user_login, text="Password:", font=("Georgia", 20), fg="white", bg="black")
password_label.grid(row=2, column=0, padx=20, pady=10)
password_entry = Entry(user_login, font=("Arial", 20), show="•")
password_entry.grid(row=2, column=1, padx=20, pady=10)
login_button = Button(user_login, text="Login", font=("Goudy Old Style", 16), fg="white", bg="black",command=login_user)
login_button.grid(row=3, column=0, columnspan=3, padx=20, pady=10)
register_message_label = Label(user_login, text="IF YOU DON'T HAVE ACCOUNT , PLEASE REGISTER", font=("Bell MT", 12), fg="white", bg="black")
register_message_label.grid(row=4, column=0, columnspan=2, padx=20, pady=5)
registration_button = Button(user_login, text="Register", font=("Goudy Old Style", 16), fg="white", bg="black",command=switch_to_user_registration)
registration_button.grid(row=5, column=0, columnspan=2, padx=20, pady=10)
home_button = Button(user_login, text="Back to Home", font=("Goudy Old Style", 16), fg="white", bg="black", command=homepage)
home_button.grid(row=6, column=0, columnspan=2, padx=20, pady=10)


# User Registration Widgets
user_registration_heading_label = Label(user_registration, text="User Registration", font=("Cooper Black", 40), fg="white", bg="black")
user_registration_heading_label.grid(row=0, column=0, columnspan=2, pady=20)
user_username_label = Label(user_registration, text="Username:", font=("Georgia", 18), fg="white", bg="black")
user_username_label.grid(row=1, column=0, pady=10)
user_username_entry = Entry(user_registration, font=("Arial", 18), width=30)
user_username_entry.grid(row=1, column=1, pady=10)
user_password_label = Label(user_registration, text="Password:", font=("Georgia", 18), fg="white", bg="black")
user_password_label.grid(row=2, column=0, pady=10)
user_password_entry = Entry(user_registration, font=("Arial", 18), width=30, show="•")
user_password_entry.grid(row=2, column=1, pady=10)
user_address_label = Label(user_registration, text="Address:", font=("Georgia", 18), fg="white", bg="black")
user_address_label.grid(row=3, column=0, pady=10)
user_address_entry = Entry(user_registration, font=("Arial", 18), width=30)
user_address_entry.grid(row=3, column=1, pady=10)
user_phone_label = Label(user_registration, text="Phone Number:", font=("Georgia", 18), fg="white", bg="black")
user_phone_label.grid(row=4, column=0, pady=10)
user_phone_entry = Entry(user_registration, font=("Arial", 18), width=30)
user_phone_entry.grid(row=4, column=1, pady=10)
register_button = Button(user_registration, text="Register", font=("Goudy Old Style", 16), fg="white",bg="black", command=register_user)
register_button.grid(row=5, column=0, columnspan=2, pady=10)
user_home_button = Button(user_registration, text="Back to Home", font=("Goudy Old Style", 16), fg="white", bg="black", command=homepage)
user_home_button.grid(row=6, column=0, columnspan=2, padx=20, pady=10)

# Admin Login Widgets
admin_login_heading_label = Label(admin_login, text="Doctor Login", font=("Cooper Black", 40), fg="white", bg="black")
admin_login_heading_label.grid(row=0, column=0, columnspan=2, pady=20)
admin_login_username_label = Label(admin_login, text="Username:", font=("Georgia", 18), fg="white", bg="black")
admin_login_username_label.grid(row=1, column=0, pady=10)
admin_login_username_entry = Entry(admin_login, font=("Arial", 18), width=30)
admin_login_username_entry.grid(row=1, column=1, pady=10)
admin_login_password_label = Label(admin_login, text="Password:", font=("Georgia", 18), fg="white", bg="black")
admin_login_password_label.grid(row=2, column=0, pady=10)
admin_login_password_entry = Entry(admin_login, font=("Arial", 18), width=30, show="•")
admin_login_password_entry.grid(row=2, column=1, pady=10)
admin_login_button = Button(admin_login, text="Login", font=("Goudy Old Style", 16), fg="white",bg="black", command=login_admin)
admin_login_button.grid(row=3, column=0, columnspan=2, pady=10)
admin_register_message_label = Label(admin_login, text="IF YOU DON'T HAVE ACCOUNT , PLEASE REGISTER", font=("Bell MT", 12), fg="white", bg="black")
admin_register_message_label.grid(row=4, column=0, columnspan=2, padx=20, pady=5)
admin_registration_button = Button(admin_login, text="Register", font=("Goudy Old Style", 16), fg="white", bg="black",command=switch_to_admin_registration)
admin_registration_button.grid(row=5, column=0, columnspan=2, padx=20, pady=10)
admin_home_button = Button(admin_login, text="Back to Home", font=("Goudy Old Style", 16), fg="white", bg="black", command=homepage)
admin_home_button.grid(row=6, column=0, columnspan=2, padx=20, pady=10)

# Admin Registration Widgets
admin_registration_heading_label = Label(admin_registration, text="Admin Registration", font=("Cooper Black", 40), fg="white", bg="black")
admin_registration_heading_label.grid(row=0, column=0, columnspan=2, pady=20)
admin_username_label = Label(admin_registration, text="Username:", font=("Georgia", 18), fg="white", bg="black")
admin_username_label.grid(row=1, column=0, pady=10)
admin_username_entry = Entry(admin_registration, font=("Arial", 18), width=30)
admin_username_entry.grid(row=1, column=1, pady=10)
admin_password_label = Label(admin_registration, text="Password:", font=("Georgia", 18), fg="white", bg="black")
admin_password_label.grid(row=2, column=0, pady=10)
admin_password_entry = Entry(admin_registration, font=("Arial", 18), width=30, show="•")
admin_password_entry.grid(row=2, column=1, pady=10)
admin_address_label = Label(admin_registration, text="Address:", font=("Georgia", 18), fg="white", bg="black")
admin_address_label.grid(row=3, column=0, pady=10)
admin_address_entry = Entry(admin_registration, font=("Arial", 18), width=30)
admin_address_entry.grid(row=3, column=1, pady=10)
admin_phone_label = Label(admin_registration, text="Phone Number:", font=("Georgia", 18), fg="white", bg="black")
admin_phone_label.grid(row=4, column=0, pady=10)
admin_phone_entry = Entry(admin_registration, font=("Arial", 18), width=30)
admin_phone_entry.grid(row=4, column=1, pady=10)
admin_register_button = Button(admin_registration, text="Register", font=("Goudy Old Style", 16), fg="white", bg="black",command=register_admin)
admin_register_button.grid(row=5, column=0, columnspan=2, pady=10)
admin_home_button = Button(admin_registration, text="Back to Home", font=("Goudy Old Style", 16), fg="white", bg="black", command=homepage)
admin_home_button.grid(row=6, column=0, columnspan=2, padx=20, pady=10)

# User Details Widgets
appointment_button = Button(user_details, text="Book Appointment", font=("Goudy Old Style", 16), fg="white",bg="black", command=switch_to_appointment_frame)
appointment_button.grid(row=0, column=0,pady=10)
feedback_button = Button(user_details, text="Feedback from Doctor", font=("Goudy Old Style", 16), fg="white", bg="black",command=fetch_patient_feedback)
feedback_button.grid(row=0, column=2, pady=10)
button = Button(user_details, text="Back", font=("Goudy Old Style", 16), fg="white", bg="black",command=homepage)
button.grid(row=2, column=1, pady=10)

#.....Patient Details
user_details_heading_label = Label(frame10, text="Patient Details", font=("Cooper Black", 40), fg="white", bg="black")
user_details_heading_label.grid(row=1, column=0, columnspan=2, pady=20)
user_height_label = Label(frame10, text="Height:", font=("Georgia", 18), fg="white", bg="black")
user_height_label.grid(row=2, column=0, pady=10)
user_height_entry = Entry(frame10, font=("Arial", 18), width=30)
user_height_entry.grid(row=2, column=1, pady=10)
user_weight_label = Label(frame10, text="Weight:", font=("Georgia", 18), fg="white", bg="black")
user_weight_label.grid(row=3, column=0, pady=10)
user_weight_entry = Entry(frame10, font=("Arial", 18), width=30)
user_weight_entry.grid(row=3, column=1, pady=10)
user_temp_label = Label(frame10, text="Temp:", font=("Georgia", 18), fg="white", bg="black")
user_temp_label.grid(row=4, column=0, pady=10)
user_temp_entry = Entry(frame10, font=("Arial", 18), width=30)
user_temp_entry.grid(row=4, column=1, pady=10)
user_BP_label = Label(frame10, text="BP:", font=("Georgia", 18), fg="white", bg="black")
user_BP_label.grid(row=5, column=0, pady=10)
user_BP_entry = Entry(frame10, font=("Arial", 18), width=30)
user_BP_entry.grid(row=5, column=1, pady=10)
pro_label = Label(frame10, text="Problem:", font=("Georgia", 18), fg="white", bg="black")
pro_label.grid(row=6, column=0, pady=10)
pro_text = Text(frame10, font=("Arial", 18), width=30, height=5)
pro_text.grid(row=6, column=1, pady=10)
save_button = Button(frame10, text="Save", font=("Goudy Old Style", 16), fg="white",bg="black", command=save_online)
save_button.grid(row=7, column=0, columnspan=2, pady=10)
button = Button(frame10, text="Back to home ", font=("Goudy Old Style", 16), fg="white",bg="black", command=homepage)
button.grid(row=8, column=0, columnspan=2, pady=10)


# Admin Dashboard Widgets
admin_dashboard_heading_label = Label(frame11, text="Dr Dashboard", font=("Cooper Black", 40), fg="white", bg="black")
admin_dashboard_heading_label.grid(row=0, column=0, columnspan=2, pady=20)
user_name_label = Label(frame11, text="Patient Name:", font=("Georgia", 18), fg="white", bg="black")
user_name_label.grid(row=1, column=0, pady=10)
user_name_entry = Entry(frame11, font=("Arial", 18), width=30)
user_name_entry.grid(row=1, column=1, pady=10)
fetch_button = Button(frame11, text="View Patient Details", font=("Goudy Old Style", 16), fg="white",bg="black", command=fetch_health_details)
fetch_button.grid(row=2, column=0, columnspan=2, pady=10)
health_details_text = Text(frame11, font=("Arial", 18), width=50, height=10)
health_details_text.grid(row=3, column=0, columnspan=2, pady=10)
feedback_label = Label(frame11, text="Provide Feedback:", font=("Georgia", 18),fg="white", bg="black")
feedback_text = Text(frame11, width=50, height=5, font=("Arial", 16))
feedback_text.grid(row=5,column=0,columnspan=2,pady=10)
submit_feedback_button = Button(frame11, text="Submit Feedback",  font=("Goudy Old Style", 16),fg="white",bg="black",command=submit_feedback,)
submit_feedback_button.grid(row=6,column=0)
backbutton=Button(frame11, text="Back To Home Page", font=("Goudy Old Style", 16),fg="white", bg="black",command=homepage)
backbutton.grid(row=6, column=1,pady=10)
back_button=Button(frame11, text="Back", font=("Goudy Old Style", 16),fg="white", bg="black",command=switch_to_admin_panel)
back_button.grid(row=7, column=0,columnspan=2,pady=10)


button = Button(admin_dashboard, text="Dash Board", font=("Goudy Old Style", 16), fg="white",bg="black", command=switch_to_frame11)
button.grid(row=1, column=0, columnspan=2, pady=10)
view_appointment_button = Button(admin_dashboard, text="View Appointments", font=("Goudy Old Style", 16), fg="white",bg="black", command=switch_to_appointment_view)
view_appointment_button.grid(row=2, column=0, columnspan=1, pady=10)

# Appointment Booking Widgets
appointment_heading_label = Label(appointment_frame, text="Book Appointment", font=("Cooper Black", 40), fg="white", bg="black")
appointment_heading_label.grid(row=0, column=0, columnspan=2, pady=20)
appointment_date_label = Label(appointment_frame, text="Appointment Date (YYYY-MM-DD):", font=("Georgia", 18), fg="white", bg="black")
appointment_date_label.grid(row=1, column=0, pady=10)
appointment_date_entry = Entry(appointment_frame, font=("Arial", 18), width=30)
appointment_date_entry.grid(row=1, column=1, pady=10)
appointment_time_label = Label(appointment_frame, text="Appointment Time (HH:MM  AM/PM):", font=("Georgia", 18), fg="white", bg="black")
appointment_time_label.grid(row=2, column=0, pady=10)
appointment_time_entry = Entry(appointment_frame, font=("Arial", 18), width=30)
appointment_time_entry.grid(row=2, column=1, pady=10)
appointment_button = Button(appointment_frame, text="Book Appointment", font=("Goudy Old Style", 16), fg="white",bg="black", command=book_appointment)
appointment_button.grid(row=3, column=0, columnspan=2, pady=10)
button=Button(appointment_frame, text="Fill the details", font=("Goudy Old Style", 16), fg="white", bg="black", command=switch_to_frame10)
button.grid(row=4, column=0, columnspan=2, padx=20, pady=10)

# Appointment Viewing Widgets
appointment_view_heading_label = Label(appointment_view_frame, text="View Appointments", font=("Cooper Black", 40), fg="white", bg="black")
appointment_view_heading_label.grid(row=0, column=0, columnspan=2, pady=20)
appointment_text = Text(appointment_view_frame, font=("Arial", 18), width=50, height=10)
appointment_text.grid(row=1, column=0, columnspan=2, pady=10)
view_appointments_button = Button(appointment_view_frame, text="Appointment List", font=("Goudy Old Style", 16), fg="white",bg="black", command=view_appointments)
view_appointments_button.grid(row=2, column=0, columnspan=2, pady=10)
admin_home_button = Button(appointment_view_frame, text="Back to Dashboard", font=("Goudy Old Style", 16), fg="white", bg="black", command=switch_to_admin_panel)
admin_home_button.grid(row=3, column=0, columnspan=2, padx=20, pady=10)
appointment_view_home_button = Button(appointment_view_frame, text="Back to Home", font=("Goudy Old Style", 16), fg="white", bg="black", command=homepage)
appointment_view_home_button.grid(row=4, column=0, columnspan=2, padx=20, pady=10)

homepage()

window.mainloop()
